### Imports

In [ ]:
import os
import numpy as np
import pandas as pd

### Inputs

In [ ]:
fusion_output_df = pd.read_csv("../outputs/fusion_output.csv")
fusion_output_confidence = fusion_output_df['confidence'].values

In [ ]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

### Kalman Filter

In [ ]:
class KalmanFilter:
    def __init__(self, process_variance=1e-3, measurement_variance=1e-2, initial_estimate=0.5):
        self.x = initial_estimate       
        self.P = 1.0                    
        self.Q = process_variance       
        self.R = measurement_variance   

    def update(self, measurement):
        self.P += self.Q
        K = self.P / (self.P + self.R)
        self.x += K * (measurement - self.x)
        self.P *= (1 - K)
        return self.x

In [ ]:
def apply_kalman(confidences):
    kf = KalmanFilter()
    smoothed = []
    for z in confidences:
        smoothed.append(kf.update(z))
    return np.array(smoothed)

### Apply

In [ ]:
conf = apply_kalman(fusion_output_confidence)

In [ ]:
tau = 0.5
cls = (conf >= tau).astype(int)

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

In [ ]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

for i, row in enumerate(edges_input):
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls[i]:<8} {CLASS_NAMES[cls[i]]:<8} {conf[i]:<10.2f}")

### Save

In [ ]:
kalman_output = pd.DataFrame({
    'class': cls,
    'confidence': conf
})

os.makedirs("../outputs", exist_ok=True)
kalman_output.to_csv("../outputs/kalman_output.csv", index=False)

print("Filtro de Kalman salvo em ../outputs/kalman_output.csv")